## Initialize Simulation Environment

In [1]:
#!/usr/bin/env python

"""demo_simulate_nyu_finger_double

Simple demo showing how the simulation setup works.

License: BSD 3-Clause License
Copyright (C) 2018-2021, New York University , Max Planck Gesellschaft
Copyright note valid unless otherwise stated in individual files.
All rights reserved.
"""

import time
from robot_properties_nyu_finger.config import NYUFingerDoubleConfig0, NYUFingerDoubleConfig1
from robot_properties_nyu_finger.wrapper import NYUFingerRobot
from bullet_utils.env import BulletEnv
import numpy as np
import pinocchio as pin

pybullet build time: Aug  2 2021 16:06:49


In [2]:
# ! Create a Pybullet simulation environment before any robots !
env = BulletEnv()

# Create a robot instance. This adds the robot to the simulator as well.
config0 = NYUFingerDoubleConfig0()
config1 = NYUFingerDoubleConfig1()
finger0 = NYUFingerRobot(config=config0)
finger1 = NYUFingerRobot(config=config1)

# pinocchio robot wrapper
pin_robot_0 = config0.buildRobotWrapper()
pin_robot_1 = config1.buildRobotWrapper()

# Add the robot to the env to update the internal structure of the robot
# at every simulation steps.
env.add_robot(finger0)
env.add_robot(finger1)

## Define Controller /s

In [7]:
import numpy as np
import matplotlib.pylab as plt

from dynamic_graph_head import ThreadHead, SimHead

HoldPDController 

In [8]:
from dynamic_graph_head import HoldPDController

Zero Torque Controller

In [9]:
class ZeroTorquesController:
    def __init__(self, head):
        self.head = head

        # Zero the commands.
        self.tau = np.zeros_like(head.get_sensor("joint_positions"))

    def warmup(self, thread_head):
        pass

    def run(self, thread_head):
        self.head.set_control("ctrl_joint_torques", self.tau)

Gravity Compensation Controller

In [21]:
class GravityCompensationController:
    def __init__(self, head):
        """
        Args:
            head: Instance of DGHead or SimHead.
        """
        self.head = head
        
        self.joint_positions = head.get_sensor("joint_positions")
        self.joint_velocities = head.get_sensor("joint_velocities")
        
        self.zeros_pos = np.zeros_like(self.joint_positions)
        # TODO: get the pinochhio wrapper in a better way
        self.robot = head._robot.pin_robot # not the best way, but welp it works

    def warmup(self, thread_head):
        self.base_pos = self.joint_positions.copy()
        return

    def rnea(self, q, dq, ddq):
        return pin.pinocchio_pywrap.rnea(self.robot.model, self.robot.data, q, dq, ddq)
    
    def run(self, thread_head):
        self.tau = self.rnea(self.joint_positions, self.zeros_pos, self.zeros_pos)
        print(self.tau)
        self.head.set_control("ctrl_joint_torques", self.tau)
        return

## Initialize robot with defined controller

In [22]:
head0 = SimHead(finger0, with_sliders=False) # left
head1 = SimHead(finger1, with_sliders=False) # right

hold_ctrl0 = HoldPDController(head0, 3., 0.05, False)  
hold_ctrl1 = HoldPDController(head1, 3., 0.05, False)

zero_ctrl0 = ZeroTorquesController(head0)
zero_ctrl1 = ZeroTorquesController(head1)

grav_ctrl0 = GravityCompensationController(head0)
grav_ctrl1 = GravityCompensationController(head1)

In [23]:
thread_head = ThreadHead(
    0.001, # dt.
    [hold_ctrl0, hold_ctrl1], # Safety controllers.
    {'head0': head0, 'head1':head1}, # Heads to read / write from.
    [], 
    env # Environment to step.
)

In [24]:
env.step(1000)

## Run Simulation

In [27]:
thread_head.switch_controllers([grav_ctrl0, grav_ctrl1])

[ 6.21032718e-02  1.60678858e-17 -1.12595994e-18]
[ 5.37454004e-02 -1.63188603e-18 -5.51929709e-19]


In [28]:
for i in range(1000):
    thread_head.run_main_loop(True)

[ 6.20933981e-02  1.61048450e-17 -1.12858115e-18]
[ 5.34341956e-02 -1.64708506e-18 -5.60525162e-19]
[ 6.20835246e-02  1.61418394e-17 -1.13120486e-18]
[ 5.31229081e-02 -1.66223741e-18 -5.69117065e-19]
[ 6.20736514e-02  1.61788719e-17 -1.13383129e-18]
[ 5.28115385e-02 -1.67736678e-18 -5.77703515e-19]
[ 6.20637784e-02  1.62159575e-17 -1.13646151e-18]
[ 5.25000873e-02 -1.69240458e-18 -5.86289624e-19]
[ 6.20539058e-02  1.62531111e-17 -1.13909660e-18]
[ 5.21885551e-02 -1.70737348e-18 -5.94873495e-19]
[ 6.20440334e-02  1.62903239e-17 -1.14173590e-18]
[ 5.18769426e-02 -1.72225234e-18 -6.03456897e-19]
[ 6.20341613e-02  1.63275632e-17 -1.14437710e-18]
[ 5.15652503e-02 -1.73706563e-18 -6.12038073e-19]
[ 6.20242895e-02  1.63648560e-17 -1.14702211e-18]
[ 5.12534789e-02 -1.75181281e-18 -6.20616702e-19]
[ 6.20144180e-02  1.64021934e-17 -1.14967030e-18]
[ 5.09416290e-02 -1.76649569e-18 -6.29192927e-19]
[ 6.20045468e-02  1.64395784e-17 -1.15232189e-18]
[ 5.06297011e-02 -1.78108932e-18 -6.37767980e-19]


[ 6.11566711e-02  1.97814313e-17 -1.38937466e-18]
[ 2.35783241e-02 -2.71757673e-18 -1.36019937e-18]
[ 6.11468243e-02  1.98206060e-17 -1.39215274e-18]
[ 2.32619158e-02 -2.72441712e-18 -1.36832975e-18]
[ 6.11369778e-02  1.98597646e-17 -1.39492963e-18]
[ 2.29454823e-02 -2.73114214e-18 -1.37645294e-18]
[ 6.11271316e-02  1.98989222e-17 -1.39770642e-18]
[ 2.26290243e-02 -2.73784495e-18 -1.38456227e-18]
[ 6.11172857e-02  1.99380700e-17 -1.40048248e-18]
[ 2.23125423e-02 -2.74438299e-18 -1.39266719e-18]
[ 6.11074400e-02  1.99771877e-17 -1.40325635e-18]
[ 2.19960369e-02 -2.75075595e-18 -1.40076753e-18]
[ 6.10975947e-02  2.00163020e-17 -1.40602995e-18]
[ 2.16795088e-02 -2.75701023e-18 -1.40885986e-18]
[ 6.10877496e-02  2.00554162e-17 -1.40880353e-18]
[ 2.13629585e-02 -2.76319581e-18 -1.41694131e-18]
[ 6.10779048e-02  2.00945453e-17 -1.41157815e-18]
[ 2.10463866e-02 -2.76935524e-18 -1.42500793e-18]
[ 6.10680603e-02  2.01336808e-17 -1.41435321e-18]
[ 2.07297937e-02 -2.77535416e-18 -1.43307033e-18]


[ 6.02224895e-02  2.34877803e-17 -1.65207705e-18]
[-6.51078111e-03 -2.80389134e-18 -2.09074655e-18]
[ 6.02126696e-02  2.35267312e-17 -1.65483645e-18]
[-6.82693979e-03 -2.79807012e-18 -2.09790124e-18]
[ 6.02028500e-02  2.35656853e-17 -1.65759604e-18]
[-7.14306774e-03 -2.79232580e-18 -2.10502653e-18]
[ 6.01930307e-02  2.36046339e-17 -1.66035521e-18]
[-7.45916437e-03 -2.78648101e-18 -2.11213550e-18]
[ 6.01832116e-02  2.36435920e-17 -1.66311503e-18]
[-7.7752291e-03 -2.7805299e-18 -2.1192275e-18]
[ 6.01733928e-02  2.36825509e-17 -1.66587488e-18]
[-8.09126137e-03 -2.77438537e-18 -2.12630926e-18]
[ 6.01635744e-02  2.37214903e-17 -1.66863329e-18]
[-8.40726058e-03 -2.76813284e-18 -2.13337391e-18]
[ 6.01537562e-02  2.37604250e-17 -1.67139134e-18]
[-8.72322616e-03 -2.76177572e-18 -2.14042204e-18]
[ 6.01439382e-02  2.37993702e-17 -1.67415012e-18]
[-9.03915753e-03 -2.75530953e-18 -2.14745306e-18]
[ 6.01341206e-02  2.38383171e-17 -1.67690899e-18]
[-9.35505411e-03 -2.74864730e-18 -2.15447370e-18]
[ 6

[ 5.93594258e-02  2.69157335e-17 -1.89480476e-18]
[-3.41520938e-02 -1.75717163e-18 -2.65928120e-18]
[ 5.93496309e-02  2.69547721e-17 -1.89756767e-18]
[-3.44633576e-02 -1.73868044e-18 -2.66497313e-18]
[ 5.93398363e-02  2.69938104e-17 -1.90033053e-18]
[-3.47745407e-02 -1.72010364e-18 -2.67064115e-18]
[ 5.93300420e-02  2.70328399e-17 -1.90309272e-18]
[-3.50856425e-02 -1.70143984e-18 -2.67628531e-18]
[ 5.93202480e-02  2.70718636e-17 -1.90585446e-18]
[-3.53966624e-02 -1.68252024e-18 -2.68191805e-18]
[ 5.93104542e-02  2.71108848e-17 -1.90861598e-18]
[-3.57076000e-02 -1.66336147e-18 -2.68753796e-18]
[ 5.93006608e-02  2.71499185e-17 -1.91137837e-18]
[-3.60184545e-02 -1.64404934e-18 -2.69313919e-18]
[ 5.92908676e-02  2.71889205e-17 -1.91413846e-18]
[-3.63292256e-02 -1.62458261e-18 -2.69872178e-18]
[ 5.92810747e-02  2.72279178e-17 -1.91689817e-18]
[-3.66399126e-02 -1.60504192e-18 -2.70427931e-18]
[ 5.92712821e-02  2.72669017e-17 -1.91965688e-18]
[-3.69505150e-02 -1.58542854e-18 -2.70981219e-18]


[ 5.85181096e-02  3.02785751e-17 -2.13268619e-18]
[-6.05695180e-02  2.81896214e-19 -3.06745280e-18]
[ 5.85083393e-02  3.03178492e-17 -2.13546298e-18]
[-6.08718413e-02  3.10311061e-19 -3.07113811e-18]
[ 5.84985693e-02  3.03571382e-17 -2.13824081e-18]
[-6.11740372e-02  3.38677706e-19 -3.07478703e-18]
[ 5.84887995e-02  3.03964452e-17 -2.14101990e-18]
[-6.14761054e-02  3.67159450e-19 -3.07841076e-18]
[ 5.84790301e-02  3.04357617e-17 -2.14379963e-18]
[-6.17780452e-02  3.95758223e-19 -3.08200930e-18]
[ 5.84692609e-02  3.04750909e-17 -2.14658023e-18]
[-6.20798561e-02  4.24632390e-19 -3.08559428e-18]
[ 5.84594920e-02  3.05144240e-17 -2.14936108e-18]
[-6.23815376e-02  4.53624488e-19 -3.08915443e-18]
[ 5.84497235e-02  3.05537763e-17 -2.15214327e-18]
[-6.26830891e-02  4.82563171e-19 -3.09267712e-18]
[ 5.84399552e-02  3.05931391e-17 -2.15492618e-18]
[-6.29845103e-02  5.11763779e-19 -3.09618554e-18]
[ 5.84301871e-02  3.06325156e-17 -2.15771003e-18]
[-6.32858004e-02  5.41072452e-19 -3.09966802e-18]


...

[ 5.59196548e-02  4.11471827e-17 -2.90008282e-18]
[-1.35207809e-01  1.01380423e-17 -3.06217602e-18]
[ 5.59099615e-02  4.11916320e-17 -2.90321965e-18]
[-1.35458833e-01  1.01705808e-17 -3.05800121e-18]
[ 5.59002685e-02  4.12361117e-17 -2.90635861e-18]
[-1.35709606e-01  1.02027671e-17 -3.05377974e-18]
[ 5.58905758e-02  4.12806254e-17 -2.90949997e-18]
[-1.35960126e-01  1.02346034e-17 -3.04951203e-18]
[ 5.58808833e-02  4.13252004e-17 -2.91264567e-18]
[-1.36210393e-01  1.02660928e-17 -3.04519838e-18]
[ 5.58711912e-02  4.13698168e-17 -2.91579430e-18]
[-1.36460408e-01  1.02973409e-17 -3.04084672e-18]
[ 5.58614993e-02  4.14144544e-17 -2.91894440e-18]
[-1.36710169e-01  1.03283506e-17 -3.03645720e-18]
[ 5.58518078e-02  4.14591406e-17 -2.92209794e-18]
[-1.36959677e-01  1.03590126e-17 -3.03202207e-18]
[ 5.58421165e-02  4.15038554e-17 -2.92525348e-18]
[-1.37208931e-01  1.03894327e-17 -3.02754907e-18]
[ 5.58324255e-02  4.15486263e-17 -2.92841299e-18]
[-1.37457930e-01  1.04197247e-17 -3.02304641e-18]


[ 5.50097548e-02  4.55133648e-17 -3.20821421e-18]
[-1.57652194e-01  1.21039495e-17 -2.51634745e-18]
[ 5.50000888e-02  4.55620978e-17 -3.21165370e-18]
[-1.57877893e-01  1.21129764e-17 -2.50909397e-18]
[ 5.49904231e-02  4.56108408e-17 -3.21509386e-18]
[-1.58103307e-01  1.21217710e-17 -2.50181637e-18]
[ 5.49807578e-02  4.56596453e-17 -3.21853839e-18]
[-1.58328433e-01  1.21303335e-17 -2.49451507e-18]
[ 5.49710927e-02  4.57084915e-17 -3.22198585e-18]
[-1.58553272e-01  1.21388426e-17 -2.48720344e-18]
[ 5.49614279e-02  4.57574072e-17 -3.22543825e-18]
[-1.58777823e-01  1.21471167e-17 -2.47986814e-18]
[ 5.49517634e-02  4.58063490e-17 -3.22889247e-18]
[-1.59002086e-01  1.21551574e-17 -2.47250941e-18]
[ 5.49420992e-02  4.58553445e-17 -3.23235049e-18]
[-1.59226061e-01  1.21628728e-17 -2.46512118e-18]
[ 5.49324353e-02  4.59044216e-17 -3.23581432e-18]
[-1.59449747e-01  1.21702639e-17 -2.45770386e-18]
[ 5.49227717e-02  4.59535369e-17 -3.23928082e-18]
[-1.59673145e-01  1.21774185e-17 -2.45026394e-18]


[ 5.41988325e-02  4.98187326e-17 -3.51214514e-18]
[-1.75580333e-01  1.20377540e-17 -1.84074599e-18]
[ 5.41891911e-02  4.98730739e-17 -3.51598258e-18]
[-1.7578082e-01  1.2027216e-17 -1.8321494e-18]
[ 5.41795499e-02  4.99274713e-17 -3.51982398e-18]
[-1.75980993e-01  1.20164614e-17 -1.82354624e-18]
[ 5.41699091e-02  4.99819054e-17 -3.52366796e-18]
[-1.76180853e-01  1.20054898e-17 -1.81493695e-18]
[ 5.41602686e-02  5.00364044e-17 -3.52751653e-18]
[-1.76380398e-01  1.19944495e-17 -1.80633229e-18]
[ 5.41506283e-02  5.00909965e-17 -3.53137174e-18]
[-1.76579630e-01  1.19831909e-17 -1.79772154e-18]
[ 5.41409884e-02  5.01456624e-17 -3.53523218e-18]
[-1.76778547e-01  1.19717139e-17 -1.78910509e-18]
[ 5.41313487e-02  5.02004067e-17 -3.53909818e-18]
[-1.76977149e-01  1.19600204e-17 -1.78048311e-18]
[ 5.41217094e-02  5.02552338e-17 -3.54297008e-18]
[-1.77175437e-01  1.19481106e-17 -1.77185595e-18]
[ 5.41120703e-02  5.03101245e-17 -3.54684648e-18]
[-1.77373409e-01  1.19362021e-17 -1.76323948e-18]
[ 5

[ 5.32553679e-02  5.54850131e-17 -3.91239628e-18]
[-1.93696832e-01  1.00646884e-17 -1.00659352e-18]
[ 5.32457553e-02  5.55450261e-17 -3.91663488e-18]
[-1.93865326e-01  1.00366707e-17 -9.98647455e-19]
[ 5.32361429e-02  5.56050402e-17 -3.92087349e-18]
[-1.94033481e-01  1.00085417e-17 -9.90723189e-19]
[ 5.32265308e-02  5.56650601e-17 -3.92511246e-18]
[-1.94201297e-01  9.98025023e-18 -9.82817223e-19]
[ 5.32169191e-02  5.57250909e-17 -3.92935213e-18]
[-1.94368773e-01  9.95179362e-18 -9.74929754e-19]
[ 5.32073076e-02  5.57851611e-17 -3.93359457e-18]
[-1.94535910e-01  9.92317248e-18 -9.67060913e-19]
[ 5.31976964e-02  5.58452520e-17 -3.93783844e-18]
[-1.94702708e-01  9.89438693e-18 -9.59210867e-19]
[ 5.31880855e-02  5.59053685e-17 -3.94208407e-18]
[-1.94869165e-01  9.86548626e-18 -9.51383420e-19]
[ 5.31784749e-02  5.59654683e-17 -3.94632844e-18]
[-1.95035282e-01  9.83656843e-18 -9.43585902e-19]
[ 5.31688646e-02  5.60255563e-17 -3.95057189e-18]
[-1.95201059e-01  9.80748114e-18 -9.35807735e-19]


[ 5.23722363e-02  6.10236230e-17 -4.30331572e-18]
[-2.07752924e-01  6.86953619e-18 -3.81926225e-19]
